In [5]:
%load_ext autoreload
%autoreload 2

# 41.基本的なデータを生成しよう

In [2]:
import os

data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '10_output')
master_dir = os.path.join(data_dir, '99_master')
print(input_dir)

data/0_input


In [3]:
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

# 42.入力データのチェック機構を作ろう

In [6]:
import pandas as pd

m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'

m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


In [7]:
tg_ym = '202007'
target_file = 'tbl_order_' + tg_ym + '.csv'
target_data = pd.read_csv(os.path.join(input_dir, target_file)) 

In [8]:
import datetime

max_date = pd.to_datetime(target_data['order_accept_date']).max()
min_date = pd.to_datetime(target_data['order_accept_date']).min()

max_str_date = max_date.strftime('%Y%m')
min_str_date = min_date.strftime('%Y%m')

if tg_ym == min_str_date and tg_ym == max_str_date:
    print('日付が一致しました')
else:
    raise Exception('日付が一致しません')

日付が一致しました


In [10]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds()/60

def init_train_df(trg_df):
    # 保守用店舗データの削除
    trg_df = trg_df.loc[trg_df['store_id'] != 999]
    
    trg_df = pd.merge(trg_df, m_store, on='store_id', how='left')
    trg_df = pd.merge(trg_df, m_area, on='area_cd', how='left')

    trg_df.loc[trg_df['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
    trg_df.loc[trg_df['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'

    trg_df.loc[trg_df['status'] == 0, 'status_name'] = '受付'
    trg_df.loc[trg_df['status'] == 1, 'status_name'] = 'お支払い済み'
    trg_df.loc[trg_df['status'] == 2, 'status_name'] = 'お渡し済み'
    trg_df.loc[trg_df['status'] == 9, 'status_name'] = 'キャンセル'
    
    trg_df.loc[:, 'order_date'] = pd.to_datetime(trg_df['order_accept_date']).dt.date
    
    # 配達までの時間を計算
    trg_df.loc[:, 'order_accept_datetime'] = pd.to_datetime(trg_df['order_accept_date'])
    trg_df.loc[:, 'delivered_datetime'] = pd.to_datetime(trg_df['delivered_date'])
    trg_df.loc[:, 'delta'] = trg_df[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
    
    return trg_df

target_data = init_train_df(target_data)

In [11]:
target_data.head()

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_date,order_accept_datetime,delivered_datetime,delta
0,12908733,60,C87467569,76,94830062,2020-07-01 11:00:00,2020-07-01 11:11:00,0,3120,2,駒込店,TK,東京,東京,デリバリー,お渡し済み,2020-07-01,2020-07-01 11:00:00,2020-07-01 11:11:00,11.0
1,50950435,39,C70875639,69,26090445,2020-07-01 11:00:00,2020-07-01 11:47:00,1,2112,9,杉並店,TK,東京,東京,お持ち帰り,キャンセル,2020-07-01,2020-07-01 11:00:00,2020-07-01 11:47:00,47.0
2,52095695,8,C50917463,36,90556793,2020-07-01 11:00:00,2020-07-01 11:26:00,0,4744,9,板橋店,TK,東京,東京,デリバリー,キャンセル,2020-07-01,2020-07-01 11:00:00,2020-07-01 11:26:00,26.0
3,75933159,49,C00098850,14,3956649,2020-07-01 11:00:00,2020-07-01 11:19:00,1,4144,1,浅草店,TK,東京,東京,お持ち帰り,お支払い済み,2020-07-01,2020-07-01 11:00:00,2020-07-01 11:19:00,19.0
4,9697439,16,C62815610,11,81986697,2020-07-01 11:00:00,2020-07-01 11:17:00,0,2122,2,蒲田店,TK,東京,東京,デリバリー,お渡し済み,2020-07-01,2020-07-01 11:00:00,2020-07-01 11:17:00,17.0


# 43.レポーティング（本部向け）を関数化してみよう

In [30]:
import openpyxl 
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Border, Side, Font

def get_rank_df(target_data):
    '''
    店舗のデータ作成、ランキングDFの返却
    '''
    tmp = target_data.loc[target_data['status'].isin([1, 2])]
    rank = tmp.groupby('store_id')['total_amount'].sum().sort_values(ascending=False)
    rank = pd.merge(rank, m_store, on='store_id', how='left')
    
    return rank

def get_cancel_rank_df(target_data):
    '''
    キャンセル率の計算、ランキングDFの返却
    '''
    cancel_df = pd.DataFrame()
    cancel_cnt = target_data.loc[target_data['status'] == 9].groupby('store_id')['store_id'].count()
    order_cnt = target_data.loc[target_data['status'].isin([1, 2, 9])].groupby('store_id')['store_id'].count()
    cancel_rate = cancel_cnt / order_cnt * 100
    cancel_df['cancel_rate'] = cancel_rate
    cancel_df = pd.merge(cancel_df, m_store, on='store_id', how='left')
    cancel_df = cancel_df.sort_values('cancel_rate', ascending=True)
    
    return cancel_df
    

def data_export(df, ws, row_start, col_start):
    # スタイル定義
    side = Side(style='thin', color='008080')
    border = Border(top=side, bottom=side, left=side, right=side)
    
    rows = dataframe_to_rows(df, index=False, header=True)
    
    for row_no, row in enumerate(rows, row_start):
        for col_no, value in enumerate(row, col_start):
            cell = ws.cell(row_no, col_no)
            cell.value = value
            cell.border = border
            if row_no == row_start:
                cell.fill = PatternFill(patternType='solid', fgColor='008080')
                cell.font = Font(bold=True, color='FFFFFF')

In [32]:
def make_report_hq(target_data, output_folder):
    rank = get_rank_df(target_data)
    cancel_rank = get_cancel_rank_df(target_data)
    
    # Excel出力処理
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = 'サマリーレポート（本部向け）'
    
    cell = ws.cell(1, 1)
    cell.value = f'本部向け {max_str_date}月度　サマリーレポート'
    cell.font = Font(bold=True, color='008080', size=20)
    
    cell = ws.cell(3, 2)
    cell.value = f'{max_str_date}月度　売上総額'
    cell.font = Font(bold=True, color='008080', size=20)
    
    cell = ws.cell(3, 6)
    cell.value = f"{'{:,}'.format(rank['total_amount'].sum())}"
    cell.font = Font(bold=True, color='008080', size=20)
    
    # 売上ランクを直接出力
    cell = ws.cell(5, 2)
    cell.value = f'売上ランキング'
    cell.font = Font(bold=True, color='008080', size=20)
    data_export(rank, ws, 6, 2)
    
    # キャンセル率ランキングを直接出力
    cell = ws.cell(5, 8)
    cell.value = f'キャンセル率ランキング'
    cell.font = Font(bold=True, color='008080', size=20)
    data_export(cancel_rank, ws, 6, 8)
    
    wb.save(os.path.join(output_folder, f'report_hq_{max_str_date}.xlsx'))
    wb.close()

In [54]:
# 44.レポーティング（店舗向け）をしてみよう

def get_store_rank(target_id, target_df):
    rank = get_rank_df(target_df)
    store_rank = rank.loc[rank['store_id'] == target_id].index[0] + 1
    
    return store_rank


def get_store_sale(target_id, target_df):
    rank = get_rank_df(target_df)
    store_sale = rank.loc[rank['store_id'] == target_id]['total_amount']
    
    return store_sale




def get_store_cancel_rank(target_id, target_df):
    cancel_df = get_cancel_rank_df(target_df)
    cancel_df = cancel_df.reset_index()
    store_cancel_rank = cancel_df.loc[cancel_df['store_id']==target_id].index[0] + 1
    
    return store_cancel_rank


def get_store_cancel_count(target_id, target_df):
    store_cancel_count = target_df.loc[(target_df['status']==9) & (target_df['store_id'] == target_id)].groupby('store_id')['store_id'].count()
    
    return store_cancel_count

In [56]:
def get_delivery_rank_df(target_id, target_df):
    delivery = target_df.loc[target_df['status'] == 2]
    delivery_rank = delivery.groupby('store_id')['delta'].mean().sort_values()
    delivery_rank = pd.merge(delivery_rank, m_store, on='store_id', how='left')
    
    return delivery_rank


def get_delivery_rank_store(target_id, target_df):
    delivery_rank = get_delivery_rank_df(target_id, target_df)
    store_delivery_rank = delivery_rank.loc[delivery_rank['store_id'] == target_id].index[0] + 1
    
    return store_delivery_rank

In [58]:
def make_report_store(target_data, target_id, output_folder):
    rank = get_store_rank(target_id, target_data)
    sale = get_store_sale(target_id, target_data)
    cancel_rank = get_store_cancel_rank(target_id, target_data)
    cancel_count = get_store_cancel_count(target_id, target_data)
    delivery_df = get_delivery_rank_df(target_id, target_data)
    delivery_rank = get_delivery_rank_store(target_id, target_data)
    
    store_name = m_store.loc[m_store['store_id'] == target_id]['store_name'].values[0]
    
    # Excel出力処理
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = '店舗向けレポーティング'
    
    cell = ws.cell(1, 1)
    cell.value = f'{store_name}{max_str_date}月度　サマリーレポート'
    cell.font = Font(bold=True, color='008080', size=20)
    
    cell = ws.cell(3, 2)
    cell.value = f'{max_str_date}月度　売上総額'
    cell.font = Font(bold=True, color='008080', size=20)
    
    cell = ws.cell(3, 6)
    cell.value = f"{'{:,}'.format(sale.values[0])}"
    cell.font = Font(bold=True, color='008080', size=20)
    
    cell = ws.cell(5, 2)
    cell.value = f'売上ランキング'
    cell.font = Font(bold=True, color='008080', size=16)
    
    cell = ws.cell(5, 5)
    cell.value = f'{rank}位'
    cell.font = Font(bold=True, color='008080', size=16)
    
    cell = ws.cell(6, 2)
    cell.value = f'売上データ'
    cell.font = Font(bold=True, color='008080', size=16)
    
    # 表の貼り付け
    tmp_df = target_data.loc[(target_data['store_id'] == target_id) & (target_data['status'].isin([1, 2]))]
    tmp_df = tmp_df[['order_accept_date', 'customer_id', 'total_amount', 'takeout_name', 'status_name']]
    data_export(tmp_df, ws, 7, 8)
    
    # 配達完了までの時間を直接出力
    ave_time = delivery_df.loc[delivery_df['store_id'] == target_id]['delta'].values[0]
    cell = ws.cell(5, 14)
    cell.value = f'配達完了までの時間ランキング'
    
    cell = ws.cell(5, 18)
    cell.value = f'{delivery_rank}位 平均{ave_time}分'
    cell.font = Font(bold=True, color='008080', size=16)
    
    cell = ws.cell(6, 14)
    cell.value = f'各店舗の配達時間ランク'
    cell.font = Font(bold=True, color='008080', size=16)
    
    data_export(delivery_df, ws, 7, 14)
    
    wb.save(os.path.join(output_folder, f'{target_id}_{store_name}_report_{max_str_date}.xlsx'))
    wb.close()

# 45.関数を実行し動作を確認してみよう

In [33]:
make_report_hq(target_data, output_dir)

# 46.更新に対応できる出力フォルダを作成しよう

In [36]:
def make_active_folder(targetYM):
    now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    target_output_dir_name = targetYM + '_' + now
    target_output_dir = os.path.join(output_dir, target_output_dir_name)
    os.makedirs(target_output_dir)
    print(target_output_dir_name)
    
    return target_output_dir


target_output_dir = make_active_folder(tg_ym)

202007_20211010111956


# 47.都道府県別で出力できるように出力フォルダを拡張してデータを出力しよう

In [37]:
# 本部向けレポート（出力先変更）
make_report_hq(target_data, target_output_dir)

In [39]:
m_store.head(2)

,store_id,store_name,area_cd
0,1,昭島店,TK
1,2,あきる野店,TK


In [40]:
m_area.head(2)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川


In [50]:
m_store.loc[m_store['store_id'] == 3]['area_cd'].values[0]

'TK'

In [59]:
# 各店舗向けレポート（全店舗実施）
for store_id in m_store.loc[m_store['store_id'] != 999]['store_id']:
    # narrow_areaのフォルダを作成
    area_cd = m_store.loc[m_store['store_id'] == store_id]['area_cd']
    area_name = m_area.loc[m_area['area_cd'] == area_cd.values[0]]['narrow_area'].values[0]
    target_store_output_dir = os.path.join(target_output_dir, area_name)
    os.makedirs(target_store_output_dir, exist_ok=True)
    make_report_store(target_data, store_id, target_store_output_dir)

# 48.前月のデータを動的に読み込もう

In [75]:
def make_report_hq_r2(target_data_list, output_folder):
    '''
    本部向けレポーティングデータ処理（過去月データ対応ver）
    '''
    
    wb = openpyxl.Workbook()
    
    file_date = ''
    
    for tmp in target_data_list:
        df = pd.DataFrame(tmp)
        
        df_date = pd.to_datetime(df['order_accept_date']).max()
        trg_date = df_date.strftime('%Y%m')
        
        if file_date == '':
            file_date = trg_date
  
        rank = get_rank_df(df)
        cancel_rank = get_cancel_rank_df(df)
    
        ws = wb.create_sheet(title=f'{trg_date}月度')

        cell = ws.cell(1, 1)
        cell.value = f'本部向け {max_str_date}月度　サマリーレポート'
        cell.font = Font(bold=True, color='008080', size=20)

        cell = ws.cell(3, 2)
        cell.value = f'{max_str_date}月度　売上総額'
        cell.font = Font(bold=True, color='008080', size=20)

        cell = ws.cell(3, 6)
        cell.value = f"{'{:,}'.format(rank['total_amount'].sum())}"
        cell.font = Font(bold=True, color='008080', size=20)

        # 売上ランクを直接出力
        cell = ws.cell(5, 2)
        cell.value = f'売上ランキング'
        cell.font = Font(bold=True, color='008080', size=20)
        data_export(rank, ws, 6, 2)

        # キャンセル率ランキングを直接出力
        cell = ws.cell(5, 8)
        cell.value = f'キャンセル率ランキング'
        cell.font = Font(bold=True, color='008080', size=20)
        data_export(cancel_rank, ws, 6, 8)

    # wb.remove(wb.worksheets[0])
    wb.save(os.path.join(output_folder, f'report_hq_{file_date}.xlsx'))
    wb.close()

In [76]:
def make_report_store_r2(target_data_list, target_id, output_folder):
    
    wb = openpyxl.Workbook()
    
    file_date = ''
    
    for tmp in target_data_list:
        df = pd.DataFrame(tmp)
        
        df_date = pd.to_datetime(df['order_accept_date']).max()
        trg_date = df_date.strftime('%Y%m')
        
        if file_date == '':
            file_date = trg_date 
    
        rank = get_store_rank(target_id, target_data)
        sale = get_store_sale(target_id, target_data)
        cancel_rank = get_store_cancel_rank(target_id, target_data)
        cancel_count = get_store_cancel_count(target_id, target_data)
        delivery_df = get_delivery_rank_df(target_id, target_data)
        delivery_rank = get_delivery_rank_store(target_id, target_data)

        store_name = m_store.loc[m_store['store_id'] == target_id]['store_name'].values[0]

        ws = wb.create_sheet(title=f'{trg_date}月度')

        cell = ws.cell(1, 1)
        cell.value = f'{store_name}{max_str_date}月度　サマリーレポート'
        cell.font = Font(bold=True, color='008080', size=20)

        cell = ws.cell(3, 2)
        cell.value = f'{max_str_date}月度　売上総額'
        cell.font = Font(bold=True, color='008080', size=20)

        cell = ws.cell(3, 6)
        cell.value = f"{'{:,}'.format(sale.values[0])}"
        cell.font = Font(bold=True, color='008080', size=20)

        cell = ws.cell(5, 2)
        cell.value = f'売上ランキング'
        cell.font = Font(bold=True, color='008080', size=16)

        cell = ws.cell(5, 5)
        cell.value = f'{rank}位'
        cell.font = Font(bold=True, color='008080', size=16)

        cell = ws.cell(6, 2)
        cell.value = f'売上データ'
        cell.font = Font(bold=True, color='008080', size=16)

        # 表の貼り付け
        tmp_df = target_data.loc[(target_data['store_id'] == target_id) & (target_data['status'].isin([1, 2]))]
        tmp_df = tmp_df[['order_accept_date', 'customer_id', 'total_amount', 'takeout_name', 'status_name']]
        data_export(tmp_df, ws, 7, 8)

        # 配達完了までの時間を直接出力
        ave_time = delivery_df.loc[delivery_df['store_id'] == target_id]['delta'].values[0]
        cell = ws.cell(5, 14)
        cell.value = f'配達完了までの時間ランキング'

        cell = ws.cell(5, 18)
        cell.value = f'{delivery_rank}位 平均{ave_time}分'
        cell.font = Font(bold=True, color='008080', size=16)

        cell = ws.cell(6, 14)
        cell.value = f'各店舗の配達時間ランク'
        cell.font = Font(bold=True, color='008080', size=16)

        data_export(delivery_df, ws, 7, 14)
    
    wb.remove(wb.worksheets[0])
    wb.save(os.path.join(output_folder, f'{target_id}_{store_name}_report_{file_date}.xlsx'))
    wb.close()

# 49.実行して過去データとの比較をしてみよう

In [72]:
tg_ym_old = str(int(tg_ym) - 1)
target_file = 'tbl_order_' + tg_ym_old + '.csv'
target_data_old = pd.read_csv(os.path.join(input_dir, target_file))

# 過去分を初期化
target_data_old = init_tran_df(target_data_old)

df_array = [target_data, target_data_old]

In [77]:
# フォルダの動的生成
target_output_dir = make_active_folder(tg_ym)
# 本部向けレポートR2を呼ぶ
make_report_hq_r2(df_array, target_output_dir)

202007_20211010120410


In [78]:
# 各店舗向けレポート（全店舗実施）
for store_id in m_store.loc[m_store['store_id']!=999]['store_id']:
    # narrow_areaのフォルダを作成
    area_cd = m_store.loc[m_store['store_id'] == store_id]['area_cd']
    area_name = m_area.loc[m_area['area_cd'] == area_cd.values[0]]['narrow_area'].values[0]
    target_store_output_dir = os.path.join(target_output_dir, area_name)
    os.makedirs(target_store_output_dir, exist_ok=True)
    make_report_store_r2(df_array, store_id, target_store_output_dir)

# 50.画面から実行できるようにしよう

In [83]:
from IPython.display import display, clear_output
from ipywidgets import DatePicker
import datetime

def order_by_date(val):
    clear_output()
    display(date_picker)
    
    df_array = []
    
    print('データ確認　データ準備開始・・・・')
    
    date_str = str(val['new'])
    date_dt = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    target_ym = date_dt.strftime('%Y%m')
    
    # フォルダの動的生成
    target_output_dir = make_active_folder(target_ym)
    
    # 選択された基準月のデータ確認
    target_file = 'tbl_order_' + target_ym + '.csv'
    if os.path.exists(os.path.join(input_dir, target_file)) == False:
        print(f'{target_file}が存在しません')
        return
    else:
        # データの読み込み
        df = pd.read_csv(os.path.join(input_dir, target_file))
        df = init_tran_df(df)
        df_array.append(df)
        
    # 選択された基準月の1月前があるか確認
    target_ym_old = str(int(target_ym) - 1)
    target_file = 'tbl_order_' + target_ym_old + '.csv'
    if os.path.exists(os.path.join(input_dir, target_file)) == True:
        df = pd.read_csv(os.path.join(input_dir, target_file))
        df = init_tran_df(df)
        df_array.append(df)
        
    print('データ準備完了　　　レポーティング出力開始・・・')
    
    # 本部向けレポートR2を呼ぶ
    make_report_hq_r2(df_array, target_output_dir)
    
    print('管理レポート出力完了　　各店舗のレポーティング出力開始・・・')
    
    # 各店舗向けレポート（全店舗実施）
    
    for store_id in m_store.loc[m_store['store_id']!=999]['store_id']:
        # narrow_areaのフォルダを作成
        area_cd = m_store.loc[m_store['store_id'] == store_id]['area_cd']
        area_name = m_area.loc[m_area['area_cd'] == area_cd.values[0]]['narrow_area'].values[0]
        target_store_output_dir = os.path.join(target_output_dir, area_name)
        os.makedirs(target_store_output_dir, exist_ok=True)
        make_report_store_r2(df_array, store_id, target_store_output_dir)
        
    print('処理完了しました')
    
date_picker = DatePicker(value=datetime.datetime(2020, 4, 1))
date_picker.observe(order_by_date, names='value')
print('データを0_inputフォルダにコピーした後、基準月を選択してください')
display(date_picker)

データを0_inputフォルダにコピーした後、基準月を選択してください


DatePicker(value=datetime.datetime(2020, 4, 1, 0, 0))